In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
from llama_index import (
    SimpleDirectoryReader,
    ServiceContext,
    LLMPredictor,
    StorageContext,
)
from llama_index import VectorStoreIndex, SummaryIndex, SimpleKeywordTableIndex
from llama_index.composability import ComposableGraph
from llama_index.llms import OpenAI
from llama_index.response.notebook_utils import display_response

#### Load Documents

In [ ]:
reader = SimpleDirectoryReader("../paul_graham_essay/data")
documents = reader.load_data()

#### Parse into Nodes

In [ ]:
from llama_index.node_parser import SimpleNodeParser

nodes = SimpleNodeParser.from_defaults().get_nodes_from_documents(documents)

#### Add to Docstore

In [ ]:
from llama_index.storage.kvstore.firestore_kvstore import FirestoreKVStore
from llama_index.storage.docstore.firestore_docstore import FirestoreDocumentStore
from llama_index.storage.index_store.firestore_indexstore import FirestoreIndexStore

In [ ]:
kvstore = FirestoreKVStore()

storage_context = StorageContext.from_defaults(
    docstore=FirestoreDocumentStore(kvstore),
    index_store=FirestoreIndexStore(kvstore),
)

In [ ]:
storage_context.docstore.add_documents(nodes)

#### Define Multiple Indexes

Each index uses the same underlying Node.

In [ ]:
summary_index = SummaryIndex(nodes, storage_context=storage_context)

In [ ]:
vector_index = VectorStoreIndex(nodes, storage_context=storage_context)

In [ ]:
keyword_table_index = SimpleKeywordTableIndex(nodes, storage_context=storage_context)

In [ ]:
# NOTE: the docstore still has the same nodes
len(storage_context.docstore.docs)

#### Test out saving and loading

In [ ]:
# NOTE: docstore and index_store is persisted in Firestore by default
# NOTE: here only need to persist simple vector store to disk
storage_context.persist()

In [ ]:
# note down index IDs
list_id = summary_index.index_id
vector_id = vector_index.index_id
keyword_id = keyword_table_index.index_id

In [ ]:
from llama_index.indices.loading import load_index_from_storage

kvstore = FirestoreKVStore()

# re-create storage context
storage_context = StorageContext.from_defaults(
    docstore=FirestoreDocumentStore(kvstore),
    index_store=FirestoreIndexStore(kvstore),
)

# load indices
summary_index = load_index_from_storage(storage_context=storage_context, index_id=list_id)
vector_index = load_index_from_storage(
    storage_context=storage_context, vector_id=vector_id
)
keyword_table_index = load_index_from_storage(
    storage_context=storage_context, keyword_id=keyword_id
)

#### Test out some Queries

In [ ]:
chatgpt = OpenAI(temperature=0, model="gpt-3.5-turbo")
service_context_chatgpt = ServiceContext.from_defaults(llm=chatgpt, chunk_size=1024)

In [ ]:
query_engine = summary_index.as_query_engine()
list_response = query_engine.query("What is a summary of this document?")

In [ ]:
display_response(list_response)

In [ ]:
query_engine = vector_index.as_query_engine()
vector_response = query_engine.query("What did the author do growing up?")

In [ ]:
display_response(vector_response)

In [ ]:
query_engine = keyword_table_index.as_query_engine()
keyword_response = query_engine.query("What did the author do after his time at YC?")

In [ ]:
display_response(keyword_response)